In [1]:
import pandas as pd

from sentence_transformers import SentenceTransformer, models

In [2]:
###### CREATE MODEL ######
max_seq_length = 128
train_batch_size = 64

# Load teacher model
print("Load teacher model")
teacher_model = SentenceTransformer('stsb-roberta-base-v2')

# Create student model
print("Create student model")
word_embedding_model = models.Transformer("xlm-roberta-base")

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Load teacher model
Create student model


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
#### Read Datasets ####

df1 = pd.read_csv('C:/Users/USER/Desktop/Knowledge Distillaation/1 Datasets/Compiled Datasets/EN_TL_compiled_400k.csv')
df2 = pd.read_csv('C:/Users/USER/Desktop/Knowledge Distillaation/1 Datasets/Compiled Datasets/STS_TL_400k.csv')
#df3 = pd.read_csv('datasets/translate_s2.txt', delimiter = '\t', encoding = 'unicode_escape')


In [4]:
df1

,tl,en
0,"Ayon naman kay Bayan Muna Rep. Carlos Zarate, ...","According to Bayan Muna Rep. Carlos Zarate, Ch..."
1,Ayon naman sa Philippine Embassy sa Washington...,According to the Philippine Embassy in Washing...
2,Paano maipagtatanggol ng mga ordinaryong mamam...,How do ordinary citizens defend themselves fro...
3,SM by the bay,SM by the bay
4,"MANILA, Philippines - Inirekomenda ng Departme...","Manila, Philippines - The Department of Interi..."
...,...,...
399995,"""The President will discuss with Japanese Prim...","""The President Will Discuss With Japanese Prim..."
399996,Matatandaang idiniskwalipika ng Commission on ...,The Commission on Elections (COMELEC) (COMELEC...
399997,"Dahil sa huling score, inilarawan ng Heritage ...","Due to the last score, the Heritage Foundation..."
399998,"MANILA, Philippines -- Wala umanong karapatan ...","Manila, Philippines - The Government of Puerto..."


In [5]:
df2

,s1,s2,label
0,"""Hindi ko ugali ang mamulitika; mas gusto kong...","Ito ang dineklara ni Atty. Romulo Macalintal, ...",1
1,"Ayon naman kay Bayan Muna Rep. Carlos Zarate, ...",Dating itinutulak ni Duterte ang pagbabago ng ...,0
2,Ayon naman sa Philippine Embassy sa Washington...,"Ayon sa NBI, hindi umano siyento por siyentong...",1
3,Paano maipagtatanggol ng mga ordinaryong mamam...,"""Nakasaad sa R.A. 9009 na dapat mayroong land ...",1
4,SM by the bay,Kabilang sa mga sumali sa programang ito ang m...,1
...,...,...,...
419995,2p.m. Arellano University vs Hog's Breath Cafe,Inirekomenda sa Pangulo ni Albay Rep. Joey Sal...,1
419996,Jason Davee: We rather want the senate head to...,"Sabi naman ni Eberl, gumagana pa rin naman uma...",1
419997,Ipinatupad ang liquid ban sa mga istasyon ng M...,Gumastos naman ng P11.587 milyon ang mga contr...,1
419998,Team Standings: zArellano (13-4); zSan Beda (1...,z - Final Four twice-to-beat,0


In [6]:
from sentence_transformers.datasets import ParallelSentencesDataset
from torch.utils.data import DataLoader
from sentence_transformers import SentencesDataset, losses, evaluation, readers


###### Load train sets ######

train_reader = ParallelSentencesDataset(student_model=model, teacher_model=teacher_model)
train_reader.add_dataset(df1.values.tolist())
#load_data('translate_s1.txt', encoding= 'unicode_escape')
train_dataloader = DataLoader(train_reader, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MSELoss(model=model)

In [7]:
###### Load test sets Mean Squared Error (MSE) measures ######

evaluators = []

test_mse = evaluation.MSEEvaluator(df1['en'].values.tolist(),df1['tl'].values.tolist(), teacher_model=teacher_model, batch_size=train_batch_size, show_progress_bar=True, write_csv = True)
evaluators.append(test_mse)

Batches:   0%|          | 0/6250 [00:00<?, ?it/s]

In [8]:
###### Load dev sets for Semantic Textual Similarity (STS) data ######

evaluator_sts = evaluation.EmbeddingSimilarityEvaluator(df2['s1'], df2['s2'], df2['label'], batch_size=train_batch_size, show_progress_bar=True, write_csv=True)
evaluators.append(evaluator_sts)

In [9]:
import tensorflow.compat.v1 as tf

#gpus = tf.config.experimental.list_physical_devices('GPU')
#print(gpus)

#tf.config.set_visible_devices([], 'CPU') # hide the CPU
#tf.config.set_visible_devices(gpus[0], 'GPU') # unhide potentially hidden GPU
#tf.config.set_visible_devices(gpus[1], 'GPU') # unhide potentially hidden GPU
#tf.config.set_visible_devices(gpus[2], 'GPU') # unhide potentially hidden GPU
#tf.config.set_visible_devices(gpus[3], 'GPU') # unhide potentially hidden GPU

#tf.config.get_visible_devices()


tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
tf.config.get_visible_devices()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
###### Train model ######

import datetime

#from numba import jit, cuda, numba

#@cuda.jit

#with tf.device("/gpu:3"):
with strategy.scope():
    output_path = "output/model-" + datetime.date.today().strftime("%Y-%m-%d")
    model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[-1]),
          epochs=100,
          evaluation_steps=1000,
          warmup_steps=10000,
          scheduler='warmupconstant',
          output_path=output_path,
          save_best_model=True,
          optimizer_params= {'lr': 2e-5, 'eps': 1e-6, 'correct_bias': False}
          )



C:\ProgramData\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3044 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\sentence_transformers\SentenceTransformer.py:537: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  labels = torch.tensor(labels).to(self._target_device)


Batches:   0%|          | 0/6250 [00:00<?, ?it/s]

In [ ]:
evaluators

In [ ]:
import scipy.spatial

#Corpus with example sentences
corpusEN = df3['EN'][:20].values.tolist()
corpusTL = df3['TL'][:20].values.tolist()

with strategy.scope():
    corpusEN_embeddings = model.encode(corpusEN)
    corpusTL_embeddings = model.encode(corpusTL)

In [ ]:
queries = df1['EN'][88:89].values.tolist()
query_embeddings = model.encode(queries)

In [ ]:
closest_n = 5

with strategy.scope():
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpusEN_embeddings, "cosine")[0]
    
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
    
        print("\n=======\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:\n")

        for idx, distance in results[0:closest_n]:
            print(corpusEN[idx].strip(), "(Score: %.4f)" % (1-distance))

In [ ]:
queriesTL = df1['TL'][88:89].values.tolist()
#queriesTL = ['gobyerno.']
queryTL_embeddings = model.encode(queriesTL)

In [ ]:
closest_n = 5
for query, queryTL_embedding in zip(queriesTL, queryTL_embeddings):
    distances = scipy.spatial.distance.cdist([queryTL_embedding], corpusTL_embeddings, "cosine")[0]
    
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    
    print("\n=======\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:\n")

    for idx, distance in results[0:closest_n]:
        print(corpusTL[idx].strip(), "(Score: %.4f)" % (1-distance))

### after 100 epocs of training: Query Search Result

In [ ]:
import scipy.spatial

#Corpus with example sentences
corpusEN = df3['EN'][:20].values.tolist()
corpusTL = df3['TL'][:20].values.tolist()

with strategy.scope():
    corpusEN_embeddings = model.encode(corpusEN)
    corpusTL_embeddings = model.encode(corpusTL)

In [ ]:
queries = df1['EN'][88:89].values.tolist()
query_embeddings = model.encode(queries)

In [ ]:
closest_n = 5

with strategy.scope():
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpusEN_embeddings, "cosine")[0]
    
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
    
        print("\n=======\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:\n")

        for idx, distance in results[0:closest_n]:
            print(corpusEN[idx].strip(), "(Score: %.4f)" % (1-distance))

In [ ]:
queriesTL = df1['TL'][88:89].values.tolist()
#queriesTL = ['Balita tungkol sa transportasyon, gobyerno ng Pilipinas at iba pa.']
queryTL_embeddings = model.encode(queriesTL)

In [ ]:
closest_n = 5
for query, queryTL_embedding in zip(queriesTL, queryTL_embeddings):
    distances = scipy.spatial.distance.cdist([queryTL_embedding], corpusTL_embeddings, "cosine")[0]
    
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    
    print("\n=======\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:\n")

    for idx, distance in results[0:closest_n]:
        print(corpusTL[idx].strip(), "(Score: %.4f)" % (1-distance))